In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import re
import tensorflow as tf

In [3]:
!pip install transformers

     |████████████████████████████████| 1.8MB 12.4MB/s 
     |████████████████████████████████| 2.9MB 63.9MB/s 
     |████████████████████████████████| 890kB 50.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=18746d1ec8e645d730b8d06ed9505d0453c3dcaf7c227c118ff20fa4401fcae7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [122]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [123]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


Read hate dataset and convert it into train and test

In [124]:
# df = pd.read_json('babi_new_12_train.json',orient='records')
df = pd.read_csv("fox_h.csv")
df = df.drop(columns=['Unnamed: 0'])
df.label.value_counts()

0    2186
1     870
Name: label, dtype: int64

In [125]:
df.head()

,text,label
0,barryswallows Merkel would never say NO,1
1,PostApocalypticHero Expect more and more women...,1
2,californiamojo Groping people in public wasn't...,0
3,"MikeSte Merkel, possible the only person in ch...",1
4,"scientist They know very well, no means NO! Th...",1


In [126]:
df1 = pd.read_csv('filtered_tweets.csv')
df1 = df1.drop(columns=['Unnamed: 0'])
df1.columns = ['text','label']
df1.head()
df1.label.value_counts()

In [127]:
df = df.append(df1)
df.label.value_counts()

In [128]:
df = df.sample(frac = 1) 

In [129]:
msk = np.random.rand(len(df)) < 0.8

train = df[msk]
test = df[~msk]

In [130]:
df.groupby('label')['label'].count()

label
0    2186
1     870
Name: label, dtype: int64

In [131]:
train.groupby('label')['label'].count()

label
0    1744
1     702
Name: label, dtype: int64

In [132]:
test.groupby('label')['label'].count()

label
0    442
1    168
Name: label, dtype: int64

In [133]:
train.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
test.columns = ['DATA_COLUMN', 'LABEL_COLUMN']


In [134]:
train.head()

,DATA_COLUMN,LABEL_COLUMN
0,barryswallows Merkel would never say NO,1
1,PostApocalypticHero Expect more and more women...,1
2,californiamojo Groping people in public wasn't...,0
3,"MikeSte Merkel, possible the only person in ch...",1
4,"scientist They know very well, no means NO! Th...",1


In [135]:
test.head()

,DATA_COLUMN,LABEL_COLUMN
21,davidfair @quickfix GLOCK,1
26,"Nevergofullliberal That's it pass more laws, y...",0
31,Bunnysmind when no doesn't work...it is advisa...,0
38,"Lurch47 @reallynoway1 We're armed, Rufus!",1
39,SSBN738G Do you really need a law to establish...,0


In [136]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [137]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [138]:
# methods to calculate precision recall and f1
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [139]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy'), recall_m, precision_m,f1_m])



In [ ]:
hist = model.fit(train_data, epochs=2, validation_data=validation_data)

Read movie subtitle cleansed data and pass for prediction.

In [83]:
df_bb = pd.read_csv('bad_boys.csv')
df_md = pd.read_csv('mrsDoubtfire_cleansed.csv')
df_bk = pd.read_csv('blackkklanman_cleansed.csv')

In [84]:
df_bk.head()

,Unnamed: 0,sentences
0,0,﻿Have you seen Dr. Meade?
1,1,Dr. Meade!
2,2,"Oh, can you help me find Dr. Meade?"
3,3,"God, help us."
4,4,"God, save the Confederacy!"


In [85]:
df_md.head()

,Unnamed: 0,sentences
0,0,"Figaro, Figaro, Figaro, Figaro, Figaro, Figaro..."
1,1,"Ah bravo Figaro, bravo bravissimo, ah bravo Fi..."
2,2,"Salutations, snack."
3,3,Yipe. On second thought.
4,4,Yiiipe!


In [86]:
df_bb.head()

,Unnamed: 0,0
0,0,Mike! What the hell are you doing?
1,1,"It's called driving, Marcus."
2,2,Slow down.
3,3,What? Four minutes?
4,4,Speed up.


In [87]:
pred_sents_bb = df_bb['0'].to_list()
pred_sents_bk = df_bk['sentences'].to_list()
pred_sents_md = df_md['sentences'].to_list()

In [88]:
pred_sents_bk[:5]

['\ufeffHave you seen Dr. Meade?',
 'Dr. Meade!',
 'Oh, can you help me find Dr. Meade?',
 'God, help us.',
 'God, save the Confederacy!']

In [89]:
def get_annotated_sents(pred_sents_bk, name):
  last=0
  label=0
  df_pred = pd.DataFrame(columns=['sentence','label'])
  for i in range(32,len(pred_sents_bk),32):
    tf_batch = tokenizer(pred_sents_bk[last:i], max_length=128, padding=True, truncation=True, return_tensors='tf')
    tf_outputs = model(tf_batch)
    tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
    labels = ['0','1']
    label = tf.argmax(tf_predictions, axis=1)
    label = label.numpy()
    
    for j in range(32):
      df_pred.loc[len(df_pred.index)] = [pred_sents_bk[last+j],label[j]]

    last=i

  df_pred.to_csv(name)
  return df_pred


Save output to a csv

In [90]:
get_annotated_sents(pred_sents_bk, 'blackklans_bert_fox_tweet.csv')
get_annotated_sents(pred_sents_md, 'mrsdoubtfire_bert_fox_tweet.csv')
df_bb_p = get_annotated_sents(pred_sents_bb, 'badboys_bert_fox_tweet.csv')

Combination of fox and tweets:


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
1394/1394 [==============================] - 693s 487ms/step - loss: 0.1490 - accuracy: 0.9491 - recall_m: 0.6663 - precision_m: 0.0603 - f1_m: 0.1085 - val_loss: 0.1515 - val_accuracy: 0.9502 - val_recall_m: 0.7645 - val_precision_m: 0.0697 - val_f1_m: 0.1247


Epoch 2/2
1394/1394 [==============================] - 679s 487ms/step - loss: 0.0707 - accuracy: 0.9768 - recall_m: 0.8151 - precision_m: 0.0716 - f1_m: 0.1289 - val_loss: 0.2617 - val_accuracy: 0.9440 - val_recall_m: 0.8742 - val_precision_m: 0.0795 - val_f1_m: 0.1428

Using Fox:

Epoch 1/2
154/154 [==============================] - 90s 497ms/step - loss: 0.5193 - accuracy: 0.7430 - recall_m: 0.2178 - precision_m: 0.1383 - f1_m: 0.1427 - val_loss: 0.1773 - val_accuracy: 0.9459 - val_recall_m: 0.9550 - val_precision_m: 0.2825 - val_f1_m: 0.4219


Epoch 2/2
154/154 [==============================] - 75s 485ms/step - loss: 0.0683 - accuracy: 0.9761 - recall_m: 0.9528 - precision_m: 0.2948 - f1_m: 0.4390 - val_loss: 0.0943 - val_accuracy: 0.9770 - val_recall_m: 0.9713 - val_precision_m: 0.2819 - val_f1_m: 0.4228

Using Tweets:

Epoch 1/2
1236/1236 [==============================] - 1073s 857ms/step - loss: 0.1077 - accuracy: 0.9604 - recall_m: 0.6087 - precision_m: 0.0475 - f1_m: 0.0856 - val_loss: 0.2212 - val_accuracy: 0.9445 - val_recall_m: 0.7309 - val_precision_m: 0.0520 - val_f1_m: 0.0949

Epoch 2/2
1236/1236 [==============================] - 1058s 856ms/step - loss: 0.0478 - accuracy: 0.9838 - recall_m: 0.7175 - precision_m: 0.0576 - f1_m: 0.1042 - val_loss: 0.2836 - val_accuracy: 0.9369 - val_recall_m: 0.7539 - val_precision_m: 0.0540 - val_f1_m: 0.0985